In [26]:
from pynq import Overlay, MMIO
from time import sleep

class NEC_IR_Controller:
    def __init__(self, bitfile_path):
        # Load the overlay
        self.ol = Overlay(bitfile_path)
        self.ol.download()
        
        # Configure GPIO addresses (adjust these to match your Vivado design)
        self.NEC_GPIO_BASE = 0x41200000  # Output for NEC transmission
        
        # Initialize MMIO interfaces
        self.nec_gpio = MMIO(self.NEC_GPIO_BASE, 0x10000)
        
        # Configure directions:
        self.nec_gpio.write(0x04, 0x00000000)  # All outputs for NEC
        self.nec_gpio.write(0x0C, 0x00000000)  # All outputs for button control
        
        # Timing parameters
        self.NEC_HOLD_TIME = 4  # 70ms hold for NEC frame
        self.BTN_PULSE_WIDTH = 0.05  # 50ms pulse width for button

    def send_nec_command(self, value):
        """Send a 32-bit NEC command to the PL transmitter with button trigger"""
        # Step 1: Write the NEC command
        self.nec_gpio.write(0x00, 0)
        sleep(self.BTN_PULSE_WIDTH)
        self.nec_gpio.write(0x00, value)
        
        # Step 2: Pulse the button signal
        self.nec_gpio.write(0x08, 0x1)  # Assert button
        sleep(self.BTN_PULSE_WIDTH)
        self.nec_gpio.write(0x08, 0x0)  # Deassert button
        
        # Step 3: Hold the NEC command for the required duration
        sleep(self.NEC_HOLD_TIME)
        
        # Step 4: Clear the NEC command
        self.nec_gpio.write(0x00, 0)

# Example usage:
if __name__ == "__main__":
    # Initialize with correct class name
    ir = NEC_IR_Controller(
        "/home/xilinx/jupyter_notebooks/xilinx/overlays/nec/design_1_wrapper.bit"
    )
    
    try:
        # Example 1: Send NEC code for "0" button (address 0x00, command 0x68)
        ir.send_nec_command(0x00FFE01F)
        sleep(0.1)
        ir.send_nec_command(0x00FF6897)
        sleep(0.1)
        ir.send_nec_command(0x00FF6897)
        sleep(0.1)
        ir.send_nec_command(0x00FF6897)
        sleep(0.1)
        ir.send_nec_command(0x00FF38C7)
        sleep(0.1)
        ir.send_nec_command(0x00FFA857)
        
        # Add delay between commands if sending multiple
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Clean up GPIO states
        ir.nec_gpio.write(0x00, 0)
        ir.nec_gpio.write(0x08, 0)
        print("Cleanup complete")

Cleanup complete
